# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from atlascompass import mongopass
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
# mongo = MongoClient(port=27017)

## not using the local host due to issues with other services running on the same port.Instead, using the API to the free cloud service.

In [2]:
uri = "mongodb+srv://kuromasadev:{}@clusteraprda.x7kvjsy.mongodb.net/?retryWrites=true&w=majority".format(mongopass)
# Create a new client and connect to the server
mongo = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    mongo.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collection_names = db.list_collection_names()

if collection_names:
    print("Collections in the new database:")
    for collection_name in collection_names:
        print(collection_name)
else:
    print("No collections found in the new database.")


Collections in the new database:
establishments


In [50]:
# assign the collection to a variable
establishments = db['raw_import']

In [51]:
documents = establishments.find()
import pandas as pd
document_preview = pd.DataFrame(documents)

In [12]:
from bson import ObjectId
document = establishments.find_one({"_id": ObjectId("648000bb0191e0b16a204cfe")})
column_names = list(document.keys())
print("Column Names:", column_names)

Column Names: ['_id', 'FHRSID', 'ChangesByServerID', 'LocalAuthorityBusinessID', 'BusinessName', 'BusinessType', 'BusinessTypeID', 'AddressLine1', 'AddressLine2', 'AddressLine3', 'AddressLine4', 'PostCode', 'Phone', 'RatingValue', 'RatingKey', 'RatingDate', 'LocalAuthorityCode', 'LocalAuthorityName', 'LocalAuthorityWebSite', 'LocalAuthorityEmailAddress', 'scores', 'SchemeType', 'geocode', 'RightToReply', 'Distance', 'NewRatingPending', 'meta', 'links', 'longitude', 'latitude']


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [52]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
result1 = establishments.find(query)

In [53]:
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of documents with a hygiene score of 20:", count)

Number of documents with a hygiene score of 20: 41


In [54]:
# Display the first document in the results using pprint
first_document = result1.next()
pprint(first_document)

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6490b91d87b6432c4cb298b5'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extra

In [55]:
# Create an empty list to store the documents
hyg_documents = []

In [56]:
# Iterate over the cursor and append each document to the list
for document in result1:
    hyg_documents.append(document)

In [57]:
# Convert the result to a Pandas DataFrame
hyg_score_df = pd.DataFrame(hyg_documents)

# Display the number of rows in the DataFrame
row_count = len(hyg_score_df)
print("Number of rows in the Dataframe for Locations with High Hygine score:", row_count)

Number of rows in the Dataframe for Locations with High Hygine score: 40


In [58]:
# Display the first 10 rows of the DataFrame
hyg_score_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,6490b91e87b6432c4cb29c36,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI...",FHRS,"{'longitude': '1.278721', 'latitude': '51.8575...",,4617.965824,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
1,6490b91f87b6432c4cb29f40,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,http://www.tendringdc.gov.uk/,fhsadmin@tendringdc.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': '1.15927', 'latitude': '51.789429'}",,4619.656144,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
2,6490b91f87b6432c4cb2a12e,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI...",FHRS,"{'longitude': '0.10202', 'latitude': '50.770885'}",,4620.421725,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
3,6490b91f87b6432c4cb2a13d,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,http://www.lewes-eastbourne.gov.uk/,ehealth.ldc@lewes-eastbourne.gov.uk,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI...",FHRS,"{'longitude': '0.101446', 'latitude': '50.7707...",,4620.437179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
4,6490b92187b6432c4cb2aadc,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,http://www.southend.gov.uk,EnvironmentalHealth@southend.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': '0.736349', 'latitude': '51.5414...",,4625.565258,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
5,6490b92187b6432c4cb2acfd,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,http://www.southend.gov.uk,EnvironmentalHealth@southend.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': '0.7121671', 'latitude': '51.535...",,4626.200132,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
6,6490b92687b6432c4cb2c228,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,http://www.rochford.gov.uk,customerservices@rochford.gov.uk,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI...",FHRS,"{'longitude': '0.602364', 'latitude': '51.5915...",,4632.276890,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
7,6490b92787b6432c4cb2c66c,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,http://www.sevenoaks.gov.uk/,environmental.health@sevenoaks.gov.uk,"{'Hygiene': 20, 'Structural': 5, 'ConfidenceIn...",FHRS,"{'longitude': '0.146449998021126', 'latitude':...",,4633.934041,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
8,6490b92887b6432c4cb2ce7e,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,http://www.basildon.gov.uk,ehs@basildon.gov.uk,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI...",FHRS,"{'longitude': '0.462307', 'latitude': '51.57005'}",,4636.552523,False,"{'dataSource': None, 'extractDate

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [59]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}
result2 = establishments.find(query2)

In [60]:
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query2)
print("Number of documents with London as the Local Authority and RatingValue >= 4:", count)

Number of documents with London as the Local Authority and RatingValue >= 4: 0


In [43]:
# Display the first document in the results using pprint
first_document2 = next(result2,None)
if first_document2:
    pprint(first_document2)
else:
    print("No documents found.")

No documents found.


In [37]:
# Create an empty list to store the documents
auth_documents = []

In [38]:
# Iterate over the cursor and append each document to the list
for document in result2:
    auth_documents.append(document)

In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
